# Libs

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_colwidth', None)  # Без ограничения ширины столбцов
pd.set_option('display.width', None)         # Без ограничения ширины вывода
pd.set_option('display.max_columns', None)   # Показать все столбцы

# Reading data

In [48]:
candles = pd.read_csv('../data/new/forecast_data/candles.csv')
print('candles', candles.shape)
# candles['begin'] = pd.to_datetime(candles['begin'])

candles_2 = pd.read_csv('../data/new/forecast_data/candles_2.csv')
print('candles_2', candles_2.shape)

news = pd.read_csv('../data/new/forecast_data/news.csv')
print('news', news.shape)

news_2 = pd.read_csv('../data/new/forecast_data/news_2.csv')
print('news_2', news_2.shape)


candles (24197, 7)
candles_2 (1745, 7)
news (25425, 4)
news_2 (2030, 3)


In [49]:
print(candles.ticker.unique())
candles.head(2)

['AFLT' 'ALRS' 'CHMF' 'GAZP' 'GMKN' 'LKOH' 'MAGN' 'MGNT' 'MOEX' 'MTSS'
 'NVTK' 'PHOR' 'PLZL' 'ROSN' 'RUAL' 'SBER' 'SIBN' 'T' 'VTBR']


,open,close,high,low,volume,begin,ticker
0,81.50,81.7,83.20,81.16,29755530,2020-06-19,AFLT
1,81.72,82.1,83.98,80.26,18502950,2020-06-22,AFLT


In [50]:
news.head(2)

,Unnamed: 0,publish_date,title,publication
0,25389,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов,"Тенденции в отрасли. Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов с развитых и развивающихся рынков как на операционном, так и на финансовом уровнях. Можно отметить и комфортную долговую нагрузку, высокую рентабельность, стабильный денежный поток, высокие дивиденды. Капзатраты сектора высокие, однако большая часть приходится на масштабную инвестпрограмму ""Газпрома"", направленную в т.ч. на развитие ключевого проекта ""Сила Сибири"". 2019 год сложился неблагоприятно с точки зрения долларовых цен на нефть и газ, однако компании смогли заработать на девальвации рубля и экономии на налогах за счет измененного демпфирующего механизма. Отдельно отметим сохраняющийся высокий уровень дивидендных выплат в отрасли. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Выручка, млрд долл. Ключевые события. Введение обратного акциза на нефть (демпфирующий механизм) в начале года было сопряжено со сложностями адаптации к рыночным реалиям, однако корректировка позволила компаниям заработать. Переход к новой дивидендной политике – ""Газпром"" (50% от прибыли по МСФО в 2021 г.) и ""ЛУКОЙЛ"" (приоритет дивидендам, а не выкупу своих акций). Мультипликаторы Источник: Bloomberg: ПСБ Аналитика & Стратегия Риски. Основные риски отрасли сопряжены с возможным дальнейшим ухудшением ценовой конъюнктуры и изменениями в сфере налогообложения. Средние значения денежных потоков, млн долл. Источник: Bloomberg: ПСБ Аналитика & Стратегия Рекомендации: ""Газпром"". - Согласно новой дивидендной политике, ""Газпром"" планирует в течение 3 лет довести коэффициент дивидендных выплат до 50%, благодаря чему дивидендная доходность может превысить 10%. - Комфортная долговая нагрузка, которая не будет препятствовать уменьшению дивидендных выплат. - Достаточный для выплат дивидендов акционерам объем FCF и отсутствие дальнейшего наращивания инвестпрограммы. Динамика цен на нефть и газ, долл./мБТЕ Источник: Bloomberg, ПСБ Аналитика & Стратегия"
1,25388,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на внутренний рынок,"Тенденции в отрасли. Ключевые российские сталелитейные компании сохраняют конкурентоспособность на операционном и финансовом уровне относительно аналогов. Высокая рентабельность сочетается с низкой долговой нагрузкой, стабильными денежными потоками и высокой дивидендной доходностью. Прошедший год был неблагоприятным с точки зрения ценовой конъюнктуры, но высокая премия на внутреннем рынке сыграла в плюс, поэтому существенно просел экспорт. Учитывая циклический характер развития отрасли и замедление промышленного производства, финансовые результаты остались на уровне прошлого года или ухудшились, что оказало давление на котировки. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Funds form Operation, млрд долл. Корпоративные события. Из отраслевых событий отметим введение пошлин на импорт г/к проката, а также запрет на экспорт лома. В период до 2023г. рост спроса на металлургическую продукцию должна обеспечить реализация национальных проектов, которые позволят увеличить ежегодный выпуск стали на 4%. Угрозу дивидендам не несут инвестпрограммы компаний, которые остались без изменения. Из корпоративных событий центральной темой года стала сделка по продажа американского подразделения ТМК. Мультипликаторы Источник: Bloomberg Риски. Условия на рынке сырья складываются неплохо, но сырьевой сегмент уже не способен перекрывать по рентабельности стальной. Внешние условия могут негативно повлиять на FCF и скажутся на снижении дивидендных выплат. Средние значения денежных потоков, млн долл. Источник: ММК, НЛМК, ""Северсталь"", Evraz, ПСБ Аналитика&Стратегия Рекомендации: ММК. - Долговая нагрузка самая низкая среди российских металлургов. Объем денежных средств полностью покрывает 

In [41]:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import yake
from typing import List, Dict, Tuple
import warnings
# warnings.filterwarnings('ignore')

# feature enginearing

In [ ]:
class TickerRAGSystem:
    def __init__(self, ticker_mapping: Dict[str, str]):
        self.ticker_mapping = ticker_mapping
        self.company_knowledge_base = self._build_knowledge_base()
        
        try:
            self.nlp = spacy.load("ru_core_news_sm")
            print("✅ Spacy загружен")
        except OSError:
            print("⚠️ Spacy недоступен")
            self.nlp = None
            
        self.sentence_model = None
        try:
            from sentence_transformers import SentenceTransformer
            self.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
            print("✅ SentenceTransformers загружен")
        except ImportError as e:
            print(f"⚠️ SentenceTransformers недоступен: {e}")
        except Exception as e:
            print(f"⚠️ Ошибка загрузки SentenceTransformers: {e}")
        
        try:
            self.kw_extractor = yake.KeywordExtractor(lan="ru", n=2, dedupLim=0.8, top=10)
            print("✅ YAKE инициализирован")
        except Exception as e:
            print(f"⚠️ Ошибка инициализации YAKE: {e}")
            self.kw_extractor = None
        
        self._vectorize_knowledge_base()
    
    def _build_knowledge_base(self) -> Dict[str, Dict]:
        """Строит базу знаний о компаниях"""
        knowledge_base = {}
        
        industry_descriptions = {
            'нефтегазовая': [
                'нефть', 'газ', 'скважина', 'месторождение', 'трубопровод', 'нефтедобыча',
                'нефтепереработка', 'нефтехимия', 'нефтяная компания', 'газовая компания',
                'нефтепродукты', 'бензин', 'дизель', 'топливо', 'энергоносители', 'углеводороды',
                'нефтяник', 'газодобыча', 'нефтепровод', 'газопровод', 'нефтебаза', 'нефтетерминал',
                'нефтесервис', 'буровая', 'нефтяное месторождение', 'газовое месторождение',
                'нефтегазовый сектор', 'энергетическое сырье', 'нефтегазовая отрасль',
                'нефтегазовый комплекс', 'добыча нефти', 'добыча газа', 'переработка нефти',
                'нефтехимическое производство', 'нефтепродуктообеспечение', 'нефтеснабжение'
            ],
            
            'банковская': [
                'банк', 'кредит', 'вклад', 'ипотека', 'финансы', 'расчетный счет',
                'банковские услуги', 'кредитная организация', 'финансовая группа', 'банкинг',
                'депозит', 'кредитование', 'ипотечное кредитование', 'потребительский кредит',
                'банковская карта', 'платежная система', 'денежные переводы', 'банковские операции',
                'финансовые услуги', 'инвестиционный банк', 'коммерческий банк', 'центральный банк',
                'банковский сектор', 'финансовый сектор', 'банковская система', 'кредитный портфель',
                'депозитный портфель', 'банковские активы', 'банковские пассивы', 'банковская ликвидность',
                'банковский надзор', 'банковское регулирование', 'финансовая отчетность банка'
            ],
            
            'металлургическая': [
                'металл', 'сталь', 'алюминий', 'никель', 'медь', 'руда',
                'металлургия', 'черная металлургия', 'цветная металлургия', 'сталелитейная',
                'металлопрокат', 'прокат', 'металлообработка', 'горнодобывающая',
                'металлургический комбинат', 'сталеплавильный', 'металлургический завод',
                'добыча руды', 'обогащение руды', 'выплавка металла', 'производство стали',
                'производство алюминия', 'производство меди', 'металлургическая продукция',
                'металлургический комплекс', 'металлургическая отрасль', 'сталеварение',
                'металлургические мощности', 'металлургическое производство', 'металлургические активы'
            ],
            
            'ритейл': [
                'магазин', 'розница', 'продажи', 'сеть', 'торговля', 'ритейлер',
                'розничная торговля', 'торговая сеть', 'сеть магазинов', 'розничные продажи',
                'розничный рынок', 'торговая точка', 'магазинная сеть', 'ритейл оператор',
                'розничный бизнес', 'торговый зал', 'продажа товаров', 'розничная цена',
                'розничный товарооборот', 'розничная выручка', 'розничные операции',
                'розничная коммерция', 'розничная дистрибуция', 'розничная логистика',
                'управление магазином', 'розничные стандарты', 'розничные технологии',
                'ритейл платформа', 'розничный клиент', 'розничный покупатель'
            ],
            
            'телекоммуникации': [
                'связь', 'интернет', 'телефон', 'мобильный', 'телеком', 'сотовый',
                'телекоммуникации', 'оператор связи', 'мобильная связь', 'сотовая связь',
                'телефонная связь', 'интернет связь', 'телекоммуникационный', 'коммуникации',
                'телеком оператор', 'мобильный оператор', 'сетевой оператор', 'телеком услуги',
                'телеком инфраструктура', 'телеком оборудование', 'телеком сети',
                'цифровая связь', 'мобильные технологии', 'телеком рынок', 'телеком индустрия',
                'телеком сектор', 'телеком компании', 'телеком услуги связи', 'телеком тарифы',
                'телеком абоненты', 'телеком трафик', 'телеком выручка'
            ],
            
            'транспорт': [
                'авиаперевозки', 'аэропорт', 'рейс', 'транспорт', 'логистика',
                'перевозки', 'транспортные услуги', 'грузоперевозки', 'пассажирские перевозки',
                'транспортная компания', 'транспортная инфраструктура', 'транспортные средства',
                'транспортный оператор', 'транспортная логистика', 'транспортные коридоры',
                'транспортные потоки', 'транспортные тарифы', 'транспортные расходы',
                'транспортное сообщение', 'транспортная доступность', 'транспортная сеть',
                'транспортные узлы', 'транспортные хабы', 'транспортная система',
                'транспортное обеспечение', 'транспортные мощности', 'транспортные активы'
            ],
            
            'горнодобывающая': [
                'шахта', 'рудник', 'добыча', 'полезные ископаемые', 'горнодобывающая',
                'добывающая промышленность', 'горная промышленность', 'добыча полезных ископаемых',
                'горнодобывающее предприятие', 'горнодобывающий комплекс', 'добывающая отрасль',
                'горные работы', 'добыча руды', 'добыча угля', 'добыча золота', 'добыча алмазов',
                'горное дело', 'горная техника', 'горное оборудование', 'горные разработки',
                'добывающие активы', 'добывающие мощности', 'добывающие операции',
                'горнодобывающий сектор', 'добывающая компания', 'горнодобывающие работы',
                'горнодобывающая техника', 'горнодобывающее производство'
            ],
            
            'IT': [
                'интернет', 'технологи', 'социальные сети', 'поиск', 'IT',
                'информационные технологии', 'программное обеспечение', 'IT компании',
                'цифровые технологии', 'IT услуги', 'IT решения', 'IT индустрия',
                'IT сектор', 'IT рынок', 'IT продукты', 'IT разработка', 'IT инфраструктура',
                'IT сервисы', 'IT платформы', 'IT бизнес', 'IT инвестиции', 'IT инновации',
                'цифровая экономика', 'цифровая трансформация', 'IT стартап', 'IT предпринимательство',
                'IT компетенции', 'IT экспертиза', 'IT компетенции', 'IT специалисты'
            ],
            
            'энергетика': [
                'электроэнергия', 'энергетика', 'электростанция', 'энергосбыт', 'энергосистема',
                'энергетический комплекс', 'энергетическая компания', 'энергетические активы',
                'энергетические мощности', 'энергетические ресурсы', 'энергетический рынок',
                'энергетический сектор', 'энергетическая отрасль', 'энергетическое оборудование',
                'энергетические технологии', 'энергетические проекты', 'энергетические тарифы',
                'энергетические услуги', 'энергетическое производство', 'энергетическое потребление',
                'энергетическая эффективность', 'энергетическая безопасность', 'энергетическая стратегия'
            ],
            
            'химическая': [
                'химия', 'удобрения', 'пластмассы', 'химическая продукция', 'химическая промышленность',
                'химическое производство', 'химические продукты', 'химические вещества',
                'химические реактивы', 'химические соединения', 'химические технологии',
                'химическое оборудование', 'химические предприятия', 'химическая отрасль',
                'химический комплекс', 'химические мощности', 'химические активы',
                'химические инновации', 'химические исследования', 'химические разработки'
            ],
            
            'фармацевтическая': [
                'фармацевтика', 'лекарства', 'медицинские препараты', 'фармацевтическая промышленность',
                'фармацевтическое производство', 'фармацевтические компании', 'фармацевтические продукты',
                'фармацевтические препараты', 'фармацевтический рынок', 'фармацевтический сектор',
                'фармацевтическая отрасль', 'фармацевтические исследования', 'фармацевтические разработки',
                'фармацевтические инновации', 'фармацевтические технологии', 'фармацевтическое оборудование'
            ],
            
            'строительная': [
                'строительство', 'строительная компания', 'строительные работы', 'строительные проекты',
                'строительные материалы', 'строительная техника', 'строительная отрасль',
                'строительный комплекс', 'строительные услуги', 'строительные подряды',
                'строительные объекты', 'строительные мощности', 'строительные активы',
                'строительные инновации', 'строительные технологии', 'строительное производство'
            ],
            
            'агропромышленная': [
                'сельское хозяйство', 'агропром', 'агробизнес', 'агрохолдинг', 'агропромышленный',
                'агропромышленный комплекс', 'агропромышленная компания', 'агропромышленные активы',
                'агропромышленные проекты', 'агропромышленные технологии', 'агропромышленный сектор',
                'агропромышленная отрасль', 'агропромышленное производство', 'агропромышленные продукты'
            ],
            
            'машиностроение': [
                'машиностроение', 'машиностроительная', 'машиностроительный завод', 'машиностроительная компания',
                'машиностроительное производство', 'машиностроительная продукция', 'машиностроительные технологии',
                'машиностроительное оборудование', 'машиностроительные мощности', 'машиностроительные активы',
                'машиностроительный комплекс', 'машиностроительная отрасль', 'машиностроительный сектор'
            ]
        }
        
        for company_name, ticker in self.ticker_mapping.items():
            industry = self._detect_industry(company_name, industry_descriptions)
            
            knowledge_base[ticker] = {
                'company_name': company_name,
                'ticker': ticker,
                'industry': industry,
                'keywords': self._generate_company_keywords(company_name, industry, industry_descriptions),
                'aliases': self._generate_aliases(company_name)
            }
        
        return knowledge_base
    
    def _detect_industry(self, company_name: str, industry_descriptions: Dict) -> str:
        """Автоматически определяет отрасль компании"""
        company_lower = company_name.lower()
        
        for industry, keywords in industry_descriptions.items():
            if any(keyword in company_lower for keyword in keywords):
                return industry
        
        return 'другая'
    
    def _generate_company_keywords(self, company_name: str, industry: str, industry_descriptions: Dict) -> List[str]:
        """Генерирует ключевые слова для компании"""
        keywords = set()
        
        keywords.add(company_name.lower())
        
        words = company_name.lower().split()
        keywords.update(words)
        
        if industry in industry_descriptions:
            keywords.update(industry_descriptions[industry])
        
        specific_keywords = {
            'сбербанк': ['сбер', 'банк', 'финансы', 'кредит', 'вклады', 'ипотека', 'расчетный счет', 'денежные переводы', 'платежи', 'банковские услуги', 'sberbank'],
            'газпром': ['газ', 'трубопровод', 'голубое топливо', 'сила сибири', 'газовая отрасль', 'газодобыча', 'газпром нефть', 'международный газ', 'gazprom'],
            'лукойл': ['нефть', 'бензин', 'заправка', 'топливо', 'нефтедобыча', 'нефтепереработка', 'нефтяная компания', 'нефтепродукты', 'лк', 'lukoil'],
            'норильский никель': ['никель', 'медь', 'металл', 'палладий', 'платина', 'цветные металлы', 'горнодобывающая', 'рудник', 'норильск', 'nornickel'],
            'роснефть': ['нефть', 'нефтедобыча', 'скважина', 'нефтяные месторождения', 'нефтяная компания', 'роснефти', 'rosneft'],
            'аэрофлот': ['авиа', 'перевозки', 'рейс', 'аэропорт', 'авиакомпания', 'авиаперевозчик', 'полеты', 'самолет', 'aeroflot'],
            'московская биржа': ['биржа', 'мосбиржа', 'фондовый рынок', 'торги', 'ценные бумаги', 'индексы', 'трейдинг', 'moex'],
            'втб': ['втб банк', 'банк', 'кредитная организация', 'финансовая группа', 'банковские услуги', 'vtb'],
            'магнит': ['ритейл', 'магазин', 'сеть магазинов', 'продуктовый ритейл', 'розничная торговля', 'супермаркет', 'magnit'],
            'мтс': ['связь', 'мобильная связь', 'телеком', 'интернет', 'сотовая связь', 'телекоммуникации', 'оператор связи', 'mts'],
            'татнефть': ['нефть', 'татарстан', 'нефтедобыча', 'нефтяная компания', 'татнефти', 'tatneft'],
            'фосагро': ['удобрения', 'химия', 'агрохимия', 'минеральные удобрения', 'фосфаты', 'агропром', 'phosagro'],
            'полюс': ['золото', 'золотодобыча', 'драгоценные металлы', 'золотодобывающая', 'рудник', 'polyus'],
            'русал': ['алюминий', 'металлургия', 'алюминиевая компания', 'цветная металлургия', 'rusal'],
            'северсталь': ['сталь', 'металлургия', 'черная металлургия', 'сталелитейная', 'прокат', 'severstal'],
            'новатэк': ['газ', 'сжиженный газ', 'спг', 'газодобыча', 'ямал', 'novatek'],
            'алроса': ['алмазы', 'алмазодобыча', 'бриллианты', 'алмазная компания', 'якутия', 'alrosa'],
            
            # additional
            'яндекс': ['поиск', 'интернет', 'технологи', 'yandex', 'яндекс такси', 'яндекс маркет', 'it компания', 'поисковик'],
            'мвидео': ['магазин', 'техника', 'электроника', 'mvideo', 'бытовая техника', 'ритейл электроники'],
            'озон': ['озон', 'интернет магазин', 'e-commerce', 'онлайн торговля', 'ozon'],
            'тинькофф': ['тинькофф банк', 'банк', 'кредитные карты', 'онлайн банк', 'tinkoff'],
            'мегафон': ['связь', 'мобильный оператор', 'телеком', 'мегафона', 'megafon'],
            'детский мир': ['детские товары', 'ритейл', 'детский магазин', 'игрушки', 'детям'],
            'российские железные дороги': ['ржд', 'железная дорога', 'перевозки', 'жд перевозки', 'russian railways'],
            'роснефтегаз': ['нефтегазовый', 'энергетика', 'нефтегазовая компания'],
            'росгидро': ['гидроэнергетика', 'электростанции', 'энергетика', 'гидроэлектростанция'],
            'интер рао': ['энергетика', 'электроэнергия', 'энергосбыт', 'inter rao'],
            'росбанк': ['банк', 'банковские услуги', 'кредиты', 'росбанка'],
            'открытие': ['банк открытие', 'банковская группа', 'финансы'],
            'россельхозбанк': ['рсхб', 'сельхозбанк', 'агробизнес', 'сельское хозяйство']
        }
        
        if company_name in specific_keywords:
            keywords.update(specific_keywords[company_name])
        
        return list(keywords)
    
    def _generate_aliases(self, company_name: str) -> List[str]:
        """Генерирует альтернативные названия компании"""
        aliases = set()
        
        aliases.add(company_name.lower())
        
        english_mapping = {
            'газпром': 'gazprom',
            'лукойл': 'lukoil', 
            'сбербанк': 'sberbank',
            'роснефть': 'rosneft',
            'норильский никель': 'nornickel',
            'аэрофлот': 'aeroflot',
            'алроса': 'alrosa',
            'татнефть': 'tatneft',
            'яндекс': 'yandex',
            'мвидео': 'mvideo',
            'московская биржа': 'moscow exchange',
            'втб': 'vtb',
            'магнит': 'magnit',
            'мтс': 'mts',
            'фосагро': 'phosagro',
            'полюс': 'polyus',
            'русал': 'rusal',
            'северсталь': 'severstal',
            'новатэк': 'novatek',
            'озон': 'ozon',
            'тинькофф': 'tinkoff',
            'мегафон': 'megafon',
            'детский мир': 'detsky mir',
            'российские железные дороги': 'russian railways',
            'роснефтегаз': 'rosneftegaz',
            'росгидро': 'rushydro',
            'интер рао': 'inter rao',
            'росбанк': 'rosbank',
            'открытие': 'otkritie',
            'россельхозбанк': 'rshb'
        }
        
        if company_name in english_mapping:
            aliases.add(english_mapping[company_name])
        
        # Сокращения
        abbreviation_mapping = {
            'сбербанк': 'сбер',
            'московская биржа': 'мосбиржа',
            'норильский никель': 'норильск',
            'мвидео': 'м.видео',
            'роснефть': 'росснефть',
            'российские железные дороги': 'ржд',
            'россельхозбанк': 'рсхб',
            'тинькофф': 'тинькофф банк',
            'детский мир': 'дмир'
        }
        
        if company_name in abbreviation_mapping:
            aliases.add(abbreviation_mapping[company_name])
        
        return list(aliases)
    
    def _vectorize_knowledge_base(self):
        """Векторизует базу знаний для семантического поиска"""
        company_descriptions = []
        self.company_tickers = []
        
        for ticker, info in self.company_knowledge_base.items():
            description = f"{info['company_name']} {info['industry']} {' '.join(info['keywords'])}"
            company_descriptions.append(description)
            self.company_tickers.append(ticker)
        
        if self.sentence_model:
            try:
                self.company_embeddings = self.sentence_model.encode(company_descriptions)
                print("✅ База знаний векторизована с SentenceTransformers")
            except Exception as e:
                print(f"⚠️ Ошибка векторизации SentenceTransformers: {e}")
                self.sentence_model = None
        
        self.tfidf_vectorizer = TfidfVectorizer(max_features=1000)
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(company_descriptions)
        print("✅ База знаний векторизована с TF-IDF")
    
    def retrieve_relevant_companies(self, news_text: str, top_k: int = 5) -> List[Tuple[str, float]]:
        """Извлекает релевантные компании с помощью семантического поиска"""
        
        if self.sentence_model:
            try:
                news_embedding = self.sentence_model.encode([news_text])
                similarities = cosine_similarity(news_embedding, self.company_embeddings)[0]
                relevant_indices = np.argsort(similarities)[::-1][:top_k]
                
                results = []
                for idx in relevant_indices:
                    if similarities[idx] > 0.3:
                        results.append((self.company_tickers[idx], similarities[idx]))
                
                if results:
                    return results
            except Exception as e:
                print(f"⚠️ Ошибка SentenceTransformers поиска: {e}")
                self.sentence_model = None
        
        try:
            news_vector = self.tfidf_vectorizer.transform([news_text])
            similarities = cosine_similarity(news_vector, self.tfidf_matrix)[0]
            relevant_indices = np.argsort(similarities)[::-1][:top_k]
            
            results = []
            for idx in relevant_indices:
                if similarities[idx] > 0.1:
                    results.append((self.company_tickers[idx], similarities[idx]))
            
            return results
        except Exception as e:
            print(f"⚠️ Ошибка TF-IDF поиска: {e}")
            return []
    
    def extract_keywords_with_yake(self, text: str) -> List[str]:
        """Извлекает ключевые слова из текста с обработкой ошибок"""
        if not self.kw_extractor:
            return []
            
        try:
            keywords = self.kw_extractor.extract_keywords(text)
            
            processed_keywords = []
            for item in keywords:
                if isinstance(item, tuple) and len(item) >= 2:
                    score = item[0]
                    keyword = item[1]
                    
                    if isinstance(score, (int, float)) and score < 0.5:
                        processed_keywords.append(keyword.lower())
                elif isinstance(item, str):
                    processed_keywords.append(item.lower())
            
            return processed_keywords
        except Exception as e:
            print(f"⚠️ Ошибка YAKE: {e}")
            return []
    
    def extract_with_ner(self, text: str) -> List[str]:
        """Извлекает компании с помощью NER"""
        if not self.nlp:
            return []
            
        try:
            doc = self.nlp(text)
            organizations = [ent.text.lower() for ent in doc.ents if ent.label_ == 'ORG']
            
            found_tickers = []
            for org in organizations:
                for ticker, info in self.company_knowledge_base.items():
                    if any(alias in org for alias in info['aliases']):
                        found_tickers.append(ticker)
            
            return list(set(found_tickers))
        except Exception as e:
            print(f"⚠️ Ошибка NER: {e}")
            return []
    
    def extract_direct_tickers(self, text: str) -> List[str]:
        """Ищет прямые упоминания тикеров"""
        try:
            ticker_pattern = r'\b([A-Z]{3,5})\b'
            matches = re.findall(ticker_pattern, text)
            
            valid_tickers = []
            for match in matches:
                if match in self.ticker_mapping.values():
                    valid_tickers.append(match)
            
            return valid_tickers
        except Exception as e:
            print(f"⚠️ Ошибка извлечения прямых тикеров: {e}")
            return []
    
    def is_financial_context(self, text: str) -> bool:
        """Проверяет финансовый контекст"""
        financial_terms = [
            'акци', 'дивиденд', 'котировк', 'курс', 'цена', 'прибыль', 'убыток',
            'выручк', 'доход', 'отчетност', 'квартал', 'рекомендац', 'аналитик',
            'инвест', 'портфель', 'рынок', 'бирж', 'сделка', 'покупк', 'продаж'
        ]
        
        text_lower = text.lower()
        return any(term in text_lower for term in financial_terms)
    
    def extract_tickers_advanced(self, news_text: str) -> List[str]:
        """Основной метод извлечения тикеров с использованием RAG"""
        if not isinstance(news_text, str) or not news_text.strip():
            return []
        
        found_tickers = set()
        news_lower = news_text.lower()
        
        # 1. Семантический поиск (RAG)
        relevant_companies = self.retrieve_relevant_companies(news_text, top_k=5)
        for ticker, score in relevant_companies:
            if score > 0.3:
                found_tickers.add(ticker)
        
        # 2. Прямое извлечение тикеров
        direct_tickers = self.extract_direct_tickers(news_text)
        found_tickers.update(direct_tickers)
        
        # 3. NER извлечение
        ner_tickers = self.extract_with_ner(news_text)
        found_tickers.update(ner_tickers)
        
        # 4. Ключевые слова и точное совпадение
        for ticker, info in self.company_knowledge_base.items():
            for alias in info['aliases']:
                if alias in news_lower:
                    found_tickers.add(ticker)
            
            for keyword in info['keywords']:
                if keyword in news_lower and len(keyword) > 3:
                    found_tickers.add(ticker)
        
        # 5. Компании в кавычках
        quoted_patterns = [r'["]([^"]+)["]', r'[«]([^»]+)[»]', r"[']([^']+)[']"]
        for pattern in quoted_patterns:
            matches = re.findall(pattern, news_text)
            for match in matches:
                match_lower = match.lower()
                for ticker, info in self.company_knowledge_base.items():
                    if any(alias in match_lower for alias in info['aliases']):
                        found_tickers.add(ticker)
        
        # 6. Фильтрация и ранжирование
        final_tickers = self._rank_and_filter_tickers(list(found_tickers), news_text)
        
        return final_tickers
    
    def _rank_and_filter_tickers(self, tickers: List[str], news_text: str) -> List[str]:
        """Ранжирует и фильтрует тикеры по релевантности"""
        if not tickers:
            return []
        
        scores = {}
        news_lower = news_text.lower()
        
        for ticker in tickers:
            score = 0
            info = self.company_knowledge_base[ticker]
            
            for alias in info['aliases']:
                score += news_lower.count(alias) * 2
            
            for keyword in info['keywords']:
                if keyword in news_lower:
                    score += 1
            
            if ticker in news_text:
                score += 3
            
            for alias in info['aliases']:
                if f'"{alias}"' in news_text or f"«{alias}»" in news_text:
                    score += 2
            
            scores[ticker] = score
        
        sorted_tickers = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        
        return [ticker for ticker, score in sorted_tickers if score >= 1][:5]
    
    def extract_tickers_with_flags(self, news_text: str) -> Dict[str, any]:
        """
        Извлекает тикеры и возвращает словарь с двумя компонентами:
        - tickers: список найденных тикеров
        - flags: словарь с флагами для каждой компании
        """
        if not isinstance(news_text, str) or not news_text.strip():
            return {'tickers': [], 'flags': {ticker: False for ticker in self.ticker_mapping.values()}}
        
        found_tickers = self.extract_tickers_advanced(news_text)
        
        flags = {}
        for ticker in self.ticker_mapping.values():
            flags[ticker] = ticker in found_tickers
        
        return {
            'tickers': found_tickers,
            'flags': flags
        }

# Функциональный интерфейс с флагами
def create_ticker_extractor_with_flags(ticker_mapping: Dict[str, str]):
    """Создает экстрактор тикеров с флагами"""
    rag_system = TickerRAGSystem(ticker_mapping)
    return rag_system.extract_tickers_with_flags

def apply_to_dataframe_with_flags(df, ticker_mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Применяет извлечение тикеров ко всему DataFrame и создает отдельные столбцы для каждой компании
    """
    extractor = create_ticker_extractor_with_flags(ticker_mapping)
    
    # Применяем функцию к каждому тексту
    results = df[text_column].apply(extractor)
    
    # Создаем столбец со списком тикеров
    df['tickers'] = results.apply(lambda x: x['tickers'])
    
    # Создаем отдельные столбцы для каждой компании
    all_tickers = list(ticker_mapping.values())
    for ticker in all_tickers:
        df[f'mentioned_{ticker}'] = results.apply(lambda x: x['flags'].get(ticker, False))
    
    return df

In [60]:
ticker_mapping = {
    'аэрофлот': 'AFLT',
    'алроса': 'ALRS', 
    'газпром': 'GAZP',
    'лукойл': 'LKOH',
    'сбербанк': 'SBER',
    'роснефть': 'ROSN',
    'норильский никель': 'GMKN',
    'московская биржа': 'MOEX',
    'втб': 'VTBR',
    'магнит': 'MGNT',
    'мтс': 'MTSS',
    'татнефть': 'TATN',
    'фосагро': 'PHOR',
    'полюс': 'PLZL',
    'русал': 'RUAL',
    'северсталь': 'CHMF',
    'новатэк': 'NVTK'
}

# Применяем к датафрейму с созданием флагов
news_train_with_flags = apply_to_dataframe_with_flags(
    df=news,
    text_column='title',  # или 'text' если есть полный текст
    ticker_mapping=ticker_mapping
)

✅ Spacy загружен
✅ SentenceTransformers загружен
✅ YAKE инициализирован
✅ База знаний векторизована с SentenceTransformers
✅ База знаний векторизована с TF-IDF


In [65]:
news_train_with_flags.to_parquet('news_train_with_flags.parquet')

In [176]:
news_train_with_flags = pd.read_parquet('news_train_with_flags.parquet')
print(news_train_with_flags.shape)

(25425, 22)


# Sentiment-analysis

In [177]:
import re
import spacy
from collections import Counter
import pymorphy3

class SourceExtractor:
    def __init__(self):
        try:
            self.nlp = spacy.load("ru_core_news_sm")
        except OSError:
            print("⚠️ Модель ru_core_news_sm не найдена")
            self.nlp = None
        
        self.morph = pymorphy3.MorphAnalyzer()
        
        self.known_sources = {
            'reuters', 'bloomberg', 'financial times', 'wall street journal', 'wsj',
            'marketwatch', 'investing.com', 'yahoo finance', 'cnbc', 'bbc',
            'forbes', 'business insider', 'seeking alpha',
            'интерфакс', 'interfax', 'риа новости', 'ria', 'тасс', 'tass',
            'рбк', 'rbc', 'ведомости', 'vedomosti', 'коммерсант', 'kommersant',
            'финам', 'finam', 'бкс', 'bcs', 
            'мосбиржа', 'moex', 'spb exchange', 'nasdaq', 'nyse', 'lse',
            'сбербанк', 'sberbank', 'втб', 'vtb', 'газпромбанк', 'gazprombank',
            'альфа-банк', 'alfabank', 'цб', 'cbr', 'центробанк',
            'минфин', 'minfin', 'росконгресс', 'roscongress'
        }
        
        self.stop_phrases = {
            'на рынке', 'компания', 'данным', 'согласно', 'по данным',
            'в компании', 'эксперт', 'аналитик', 'источник', 'сообщил',
            'заявил', 'отметил', 'подчеркнул', 'рассказал', 'информация',
            'данные', 'прогноз', 'отчет', 'исследование', 'на сайте',
            'в пресс-службе', 'пресс-служба', 'в отчете', 'в исследовании',
            'министерство', 'ведомство', 'агентство', 'служба'
        }
        
        self.stop_tokens = {
            'и', 'в', 'на', 'с', 'по', 'о', 'от', 'до', 'за', 'из', 'у', 'к',
            'а', 'но', 'или', 'же', 'бы', 'ли', 'то', 'вот', 'это', 'так',
            'как', 'что', 'для', 'при', 'под', 'над', 'перед', 'после',
            'без', 'через', 'между', 'среди', 'около', 'возле', 'вдоль',
            'ов', 'ом', 'ах', 'ями', 'ами', 'ямия', 'Reuters'
        }

    def extract_sources_advanced(self, text: str) -> str:
        """
        Умное извлечение источников с улучшенной фильтрацией
        """
        if not text or not isinstance(text, str):
            return None
        
        known_source = self._find_known_sources_strict(text)
        if known_source:
            return known_source
        
        domain_source = self._extract_domains_strict(text)
        if domain_source:
            return domain_source
        
        standard_source = self._extract_standard_sources_strict(text)
        if standard_source:
            return standard_source
        
        return None

    def _extract_standard_sources_strict(self, text: str) -> str:
        """
        Строгое извлечение источников по шаблонам
        """
        patterns = [
            r'[Ии]сточник\s*:?\s*([A-Za-zА-Яа-я0-9.-]+(?:\s+[A-Za-zА-Яа-я0-9.-]+){0,2})[\\s,.:;!?]',
            r'[Пп]о\s+данным\s+([A-Za-zА-Яа-я0-9.-]+(?:\s+[A-Za-zА-Яа-я0-9.-]+){0,2})[\\s,.:;!?]',
            r'[Сс]ообщает\s+([A-Za-zА-Яа-я0-9.-]+(?:\s+[A-Za-zА-Яа-я0-9.-]+){0,2})[\\s,.:;!?]',
        ]
        
        all_candidates = []
        
        for pattern in patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                candidate = match.group(1).strip()
                if self._is_valid_source_strict(candidate):
                    all_candidates.append(candidate)
        
        quote_patterns = [
            r'[Ии]сточник\s*:?\s*["«]([^"»]+)["»]',
            r'[Пп]о\s+данным\s+["«]([^"»]+)["»]',
            r'[Сс]огласно\s+["«]([^"»]+)["»]',
        ]
        
        for pattern in quote_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                candidate = match.group(1).strip()
                if self._is_valid_source_strict(candidate):
                    all_candidates.append(candidate)
        
        if all_candidates:
            source_counter = Counter(all_candidates)
            best_source = source_counter.most_common(1)[0][0]
            return self._normalize_source(best_source)
        
        return None

    def _find_known_sources_strict(self, text: str) -> str:
        """
        Строгий поиск известных источников
        """
        text_lower = text.lower()
        found_sources = []
        
        for source in self.known_sources:
            source_pattern = rf'\b{re.escape(source)}\b'
            if re.search(source_pattern, text_lower):
                context_pattern = rf'(?:источник|данн|сообщ|по\s+данным|согласно)\s*[^.]*?\b{re.escape(source)}\b'
                if re.search(context_pattern, text_lower, re.IGNORECASE):
                    found_sources.append(source)
        
        if found_sources:
            source_counter = Counter(found_sources)
            return source_counter.most_common(1)[0][0]
        
        return None

    def _extract_domains_strict(self, text: str) -> str:
        """
        Строгое извлечение доменов
        """
        domain_patterns = [
            r'(?:источник|сайт|сайте|на\s+сайте)\s*:?\s*(?:https?://)?(?:www\.)?([a-zA-Z0-9-]+\.[a-zA-Z]{2,})',
            r'([a-zA-Z0-9-]+\.(?:com|ru|org|net|io|info|biz|edu|gov))\b',
        ]
        
        domains = []
        for pattern in domain_patterns:
            matches = re.finditer(pattern, text.lower())
            for match in matches:
                domain = match.group(1)
                if self._is_valid_domain_strict(domain):
                    domains.append(domain)
        
        if domains:
            domain_counter = Counter(domains)
            return domain_counter.most_common(1)[0][0]
        
        return None

    def _is_valid_source_strict(self, candidate: str) -> bool:
        """
        Строгая проверка валидности источника
        """
        if not candidate or len(candidate) < 2:
            return False
        
        candidate_lower = candidate.lower()
        
        if candidate_lower in self.stop_tokens:
            return False
        
        if any(phrase in candidate_lower for phrase in self.stop_phrases):
            return False
        
        general_words = {'рынок', 'компания', 'банк', 'эксперт', 'аналитик', 'данные'}
        if candidate_lower in general_words:
            return False
        
        if candidate_lower in self.known_sources:
            return True
        
        if '.' in candidate and self._is_valid_domain_strict(candidate):
            return True
        
        if any(cyrillic in candidate for cyrillic in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'):
            if self.morph:
                try:
                    words = candidate.split()
                    if len(words) > 3:  # Слишком длинное - вероятно, не источник
                        return False
                    
                    for word in words:
                        if len(word) < 3:
                            continue
                        parsed = self.morph.parse(word)[0]
                        # Отсеиваем глаголы, прилагательные, наречия
                        if parsed.tag.POS in {'VERB', 'ADJF', 'ADJS', 'ADVB', 'PRED'}:
                            return False
                except:
                    pass
        
        if len(candidate) < 3:
            return False
        
        if candidate.isdigit():
            return False
        
        if re.match(r'^[а-яё\s]+\.', candidate_lower):
            return False
        
        return True

    def _is_valid_domain_strict(self, domain: str) -> bool:
        """
        Строгая проверка домена
        """
        if not domain or len(domain) < 4:
            return False
        
        invalid_domains = {
            'com', 'ru', 'org', 'net', 'io', 'www', 'http', 'https',
            'россия', 'рф', 'google', 'yandex'
        }
        
        if domain in invalid_domains:
            return False
        
        if not any(c.isalpha() for c in domain):
            return False
        
        return True

    def _normalize_source(self, source: str) -> str:
        """
        Нормализация названия источника
        """
        if not source:
            return source
        
        source = re.sub(r'^[«"\'(\[{\-]+|[»"\')\]\-]+$', '', source.strip())
        
        source_lower = source.lower()
        source_mapping = {
            'nasdaq.com': 'nasdaq',
            'nyse.com': 'nyse',
            'finam.ru': 'finam',
            'interfax.ru': 'interfax',
            'ria.ru': 'ria',
            'rbc.ru': 'rbc',
        }
        
        for key, value in source_mapping.items():
            if key in source_lower:
                return value
        
        # Для известных источников возвращаем нормализованное название
        for known_source in self.known_sources:
            if known_source in source_lower:
                return known_source
        
        return source

    def extract_sources_from_dataframe(self, df, text_column: str) -> pd.Series:
        """
        Извлечение источников для всего DataFrame
        """
        return df[text_column].apply(self.extract_sources_advanced)


# Обновленная функция для использования
def extract_sources_advanced(text: str) -> str:
    """
    Умное извлечение источников с улучшенной фильтрацией
    """
    extractor = SourceExtractor()
    return extractor.extract_sources_advanced(text)

In [178]:
import re
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional, Any
import warnings
warnings.filterwarnings('ignore')

class EnhancedNewsAnalyzer:
    """
    Улучшенный анализатор с расширенным поиском источников и финансового контекста
    """
    
    def __init__(self):
        print("✅ Улучшенный анализатор инициализирован")
    
    def analyze_sentiment_improved(self, text: str) -> Dict[str, Any]:
        """
        Улучшенный анализ тональности для финансовых новостей
        """
        if not text or not isinstance(text, str):
            return self._neutral_result()
        
        text_lower = text.lower()
        
        positive_indicators = {
            'выгодно': 4, 'выгодн': 3, 'преимуществ': 3, 'перспектив': 3,
            'рекомендац': 4, 'покупай': 4, 'инвестиру': 3, 'лидер': 3,
            'рекорд': 4, 'прорыв': 4, 'успешн': 3, 'эффективн': 2,
            
            'прибыль': 3, 'доход': 3, 'выручк': 3, 'дивиденд': 3,
            'рентабельност': 3, 'доходност': 3, 'профицит': 3,
            'стабильн': 2, 'комфортн': 2, 'уверен': 2,
            
            'рост': 3, 'увелич': 3, 'повыш': 3, 'улучш': 3,
            'развит': 2, 'расширен': 2, 'прогресс': 2,
            
            'позитив': 3, 'оптимизм': 3, 'перспективн': 3,
            'сильн': 2, 'успех': 3, 'достижен': 2
        }
        
        negative_indicators = {
            'проблем': 3, 'риск': 3, 'убыток': 4, 'потер': 3,
            'кризис': 4, 'опасн': 3, 'угроз': 3, 'сложност': 2,
            'трудност': 2, 'нестабильн': 3, 'волатильн': 2,
            
            'паден': 3, 'сниж': 3, 'сокращ': 3, 'уменьш': 3,
            'ухудш': 3, 'просадк': 3, 'обвал': 4,
            
            'отрицательн': 3, 'негативн': 3, 'плох': 2,
            'слаб': 2, 'критич': 3, 'неблагоприятн': 3
        }
        
        intensifiers = {
            'очень': 1.5, 'крайне': 2.0, 'сильно': 1.5, 'значительн': 1.5,
            'существенн': 1.5, 'резк': 1.5, 'катастрофич': 2.0, 'рекордн': 1.5,
            'масштабн': 1.3, 'высок': 1.2, 'больш': 1.1
        }
        
        context_phrases = {
            'positive': [
                'смотрятся выгодно', 'можно отметить', 'отдельно отметим',
                'комфортная долговая', 'высокая рентабельность', 'стабильный денежный поток',
                'высокие дивиденды', 'дивидендная доходность', 'превысить 10%'
            ],
            'negative': [
                'сложился неблагоприятно', 'сопряжено со сложностями', 
                'основные риски', 'ухудшением конъюнктуры'
            ]
        }
        
        positive_score = 0
        negative_score = 0
        
        for word, weight in positive_indicators.items():
            if word in text_lower:
                count = text_lower.count(word)
                for intensifier, multiplier in intensifiers.items():
                    if f"{intensifier} {word}" in text_lower:
                        positive_score += count * weight * multiplier
                        break
                else:
                    positive_score += count * weight
        
        for word, weight in negative_indicators.items():
            if word in text_lower:
                count = text_lower.count(word)
                for intensifier, multiplier in intensifiers.items():
                    if f"{intensifier} {word}" in text_lower:
                        negative_score += count * weight * multiplier
                        break
                else:
                    negative_score += count * weight
        
        for phrase in context_phrases['positive']:
            if phrase in text_lower:
                positive_score += 5
        
        for phrase in context_phrases['negative']:
            if phrase in text_lower:
                negative_score += 5
        
        total_score = positive_score - negative_score
        
        if total_score >= 2:
            sentiment = 'positive'
            confidence = min((total_score + 5) / 15, 0.95)
        elif total_score <= -3:
            sentiment = 'negative'
            confidence = min((abs(total_score) + 5) / 15, 0.95)
        else:
            sentiment = 'neutral'
            confidence = 0.6
        
        emotional_score = total_score / max(positive_score + negative_score + 1, 10)
        
        return {
            'sentiment': sentiment,
            'confidence': confidence,
            'emotional_score': emotional_score,
            'method': 'improved_financial'
        }
    
    def has_financial_context_enhanced(self, text: str) -> bool:
        """
        УЛУЧШЕННАЯ проверка финансового контекста
        """
        if not text:
            return False
        
        text_lower = text.lower()
        
        financial_terms = [
            'акци', 'акция', 'акций', 'дивиденд', 'котировк', 'курс', 'цена',
            'прибыль', 'убыток', 'выручк', 'доход', 'отчетност', 'квартал',
            
            'млрд', 'млн', 'долл', 'рубл', 'евро', 'процент', 'пункт',
            
            'рост', 'падени', 'инвест', 'портфель', 'рынок', 'бирж',
            'сделка', 'покупк', 'продаж', 'волатильн', 'ликвидност',
            
            'финанс', 'бюджет', 'капзатрат', 'актив', 'пассив', 'баланс',
            'выплат', 'затрат', 'издержк', 'рентабельност', 'доходност',
            
            'долгов', 'нагрузк', 'денежн', 'поток', 'fcf', 'ebitda',
            'мультипликатор', 'конъюнктур', 'налогообложен', 'эмитент',
            'эмисси', 'капитализация', 'облигац', 'купон',
            
            'кредит', 'заем', 'депозит', 'вклад', 'ипотек', 'рефинанс',
            
            'ввп', 'инфляц', 'ключев', 'ставк', 'цб', 'центробанк',
            
            'нефт', 'газ', 'энерг', 'метал', 'горнодобыва', 'телеком',
            'ритейл', 'строительств', 'транспорт', 'хими'
        ]
        
        financial_phrases = [
            'денежный поток', 'долговая нагрузка', 'чистая прибыль',
            'валовая выручка', 'операционная деятельность', 'финансовый результат',
            'отчетность по мсфо', 'дивидендная политика', 'рыночная капитализация',
            'котировки акций', 'биржевые торги', 'инвестиционный портфель',
            'финансовый анализ', 'экономический показатель', 'макроэкономическая ситуация'
        ]
        
        financial_terms_count = 0
        
        for term in financial_terms:
            if term in text_lower:
                financial_terms_count += 1
        
        for phrase in financial_phrases:
            if phrase in text_lower:
                financial_terms_count += 3
        
        financial_contexts = [
            r'\b(?:отчет|отчетность|баланс|прибыль|убыток)[^.]{0,100}',
            r'\b(?:цена|курс|котировки)[^.]{0,100}(?:акци|акций|облигац)',
            r'\b(?:дивиденд|выплата)[^.]{0,100}(?:акционер|прибыль)',
        ]
        
        for pattern in financial_contexts:
            if re.search(pattern, text_lower):
                financial_terms_count += 2
        
        return financial_terms_count >= 3
    
    def _neutral_result(self):
        return {
            'sentiment': 'neutral',
            'confidence': 0.5,
            'emotional_score': 0.0,
            'method': 'fallback'
        }
    
    def analyze_news_comprehensive(self, text: str) -> Dict[str, Any]:
        """
        Комплексный анализ новости с улучшенными алгоритмами
        """
        if not text or not isinstance(text, str):
            return {
                'sentiment': 'neutral',
                'confidence': 0.0,
                'emotional_score': 0.0,
                'has_financial_context': False,
                'method': 'fallback'
            }
        
        sentiment_result = self.analyze_sentiment_improved(text)
        has_financial_context = self.has_financial_context_enhanced(text)
        
        return {
            **sentiment_result,
            'has_financial_context': has_financial_context
        }

def extract_sources_advanced(text: str) -> str:
    """
    Умное извлечение источников с использованием NLP
    """
    extractor = SourceExtractor()
    return extractor.extract_sources_advanced(text)

def reanalyze_with_enhancements(df) -> pd.DataFrame:
    """
    Перезапускаем анализ с улучшенными алгоритмами источников и финансового контекста
    """
    analyzer = EnhancedNewsAnalyzer()
    source_extractor = SourceExtractor()
    
    print("🔄 Перезапускаем анализ с улучшенными алгоритмами...")
    
    analysis_results = df['publication'].apply(analyzer.analyze_news_comprehensive)
    
    df['sentiment'] = analysis_results.apply(lambda x: x['sentiment'])
    df['sentiment_confidence'] = analysis_results.apply(lambda x: x['confidence'])
    df['emotional_score'] = analysis_results.apply(lambda x: x['emotional_score'])
    df['analysis_method'] = analysis_results.apply(lambda x: x['method'])
    df['has_financial_context'] = analysis_results.apply(lambda x: x['has_financial_context'])
    # df['source'] = df['publication'].apply(extract_sources_advanced) # пока не рабочая история
    
    print("✅ Улучшенный анализ завершен!")
    
    return df

In [179]:
news_improved = reanalyze_with_enhancements(news_train_with_flags)

✅ Улучшенный анализатор инициализирован
🔄 Перезапускаем анализ с улучшенными алгоритмами...
✅ Улучшенный анализ завершен!


In [183]:
news_improved.to_parquet('news_features.parquet')

In [ ]:
news_improvedю

In [181]:
# news_improved['source'].value_counts()

In [ ]:
# news_improved[['publication']].head(3)

In [182]:
news_improved.drop('publication', axis=1).head(5)

,Unnamed: 0,publish_date,title,tickers,mentioned_AFLT,mentioned_ALRS,mentioned_GAZP,mentioned_LKOH,mentioned_SBER,mentioned_ROSN,mentioned_GMKN,mentioned_MOEX,mentioned_VTBR,mentioned_MGNT,mentioned_MTSS,mentioned_TATN,mentioned_PHOR,mentioned_PLZL,mentioned_RUAL,mentioned_CHMF,mentioned_NVTK,sentiment,sentiment_confidence,emotional_score,analysis_method,has_financial_context
0,25389,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов,"[GAZP, TATN, ROSN]",False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,positive,0.95,0.511278,improved_financial,True
1,25388,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на внутренний рынок,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,positive,0.95,0.411111,improved_financial,True
2,25387,2020-01-03 10:13:10,Контракты на поставку газа в Белоруссию и транзит продлены до 2021 года,"[GAZP, TATN, NVTK, ROSN]",False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,neutral,0.60,0.000000,improved_financial,True
3,25386,2020-01-03 13:26:29,"ПАО ""ФосАгро"" -Внеочередное общее собрание",[PHOR],False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,neutral,0.60,0.000000,improved_financial,False
4,25385,2020-01-03 13:44:03,"ПАО ""ММК"" - Внеочередное общее собрание- ИТОГИ",[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,neutral,0.60,0.000000,improved_financial,False


In [61]:
len(ticker_mapping)

17

In [79]:
# news_train_with_flags.head(1)

In [26]:
# print(news_train['tickers'].unique().tolist())
# news_train['tickers']